In [48]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
%%time

# Third-party imports
import os.path as op
import pandas as pd
import great_expectations as ge

# Project imports
from ta_lib.core.api import display_as_tabs, initialize_environment

# Initialization
initialize_environment(debug=False, hide_warnings=True)

Wall time: 0 ns


In [50]:
# standard code-template imports
from ta_lib.core.api import (
    create_context, get_dataframe, get_feature_names_from_column_transformer, get_package_path,
    display_as_tabs, string_cleaning, merge_info, initialize_environment,
    list_datasets, load_dataset, save_dataset
)
import ta_lib.eda.api as eda

In [51]:
from ta_lib.core.api import create_context, list_datasets, load_dataset

In [52]:
config_path = op.join('conf', 'config.yml')
context = create_context(config_path)

In [53]:
list_datasets(context)

['/raw/google',
 '/raw/product_manufacturer_list',
 '/raw/sales_data',
 '/raw/social_media_data',
 '/raw/Theme_list',
 '/raw/Theme_product_list',
 '/cleaned/orders',
 '/cleaned/product',
 '/cleaned/sales',
 '/processed/sales',
 '/train/sales/features',
 '/train/sales/target',
 '/test/sales/features',
 '/test/sales/target',
 '/score/sales/output']

In [54]:
# load datasets
google_df = load_dataset(context, 'raw/google')
pro_manu_df = load_dataset(context, 'raw/product_manufacturer_list')
sales_df = load_dataset(context, 'raw/sales_data')
soc_media_df = load_dataset(context, 'raw/social_media_data')
theme_df = load_dataset(context, 'raw/Theme_list')
theme_pro_df = load_dataset(context, 'raw/Theme_product_list')

In [55]:
# Import the eda API
import ta_lib.eda.api as eda

In [56]:
display_as_tabs([('google', google_df.shape) , ('product_manufacturer_list', pro_manu_df.shape),
                ('sales_data', sales_df.shape) , ('social_media_data', soc_media_df.shape) , ('Theme_list', theme_df.shape),
                ('Theme_product_list', theme_pro_df.shape)])

Tabs
    [0] Str(tuple, name='google')
    [1] Str(tuple, name='product_manufacturer_list...)
    [2] Str(tuple, name='sales_data')
    [3] Str(tuple, name='social_media_data')
    [4] Str(tuple, name='Theme_list')
    [5] Str(tuple, name='Theme_product_list')

In [57]:
google_df.head()

,date,platform,searchVolume,Claim_ID,week_number,year_new
0,05-01-2014,google,349,916,1,2014
1,06-01-2014,google,349,916,2,2014
2,07-01-2014,google,697,916,2,2014
3,10-01-2014,google,349,916,2,2014
4,20-01-2014,google,697,916,4,2014


In [58]:
theme_df.head()

,CLAIM_ID,Claim Name
0,0,No Claim
1,8,low carb
2,15,beans
3,16,cocoa
4,26,vanilla


In [59]:
sum1 = eda.get_variable_summary(google_df)
sum2 = eda.get_variable_summary(pro_manu_df)
sum3 = eda.get_variable_summary(sales_df)
sum4 = eda.get_variable_summary(soc_media_df)
sum5 = eda.get_variable_summary(theme_df)
sum6 = eda.get_variable_summary(theme_pro_df)

display_as_tabs([('google', sum1), ('product_manufacturer_list', sum2),
                 ('sales_data', sum3), ('social_media_data', sum4),
                ('Theme_list', sum5), ('Theme_product_list', sum6)])

elementwise comparison failed; this will raise an error in the future.



Tabs
    [0] DataFrame(name='google', value=             Variable Name...)
    [1] DataFrame(name='product_manufacturer_list..., value=           Variable Name D...)
    [2] DataFrame(name='sales_data', value=                          ...)
    [3] DataFrame(name='social_media_data', value=                 Variable ...)
    [4] DataFrame(name='Theme_list', value=           Variable Name D...)
    [5] DataFrame(name='Theme_product_list', value=           Variable Name D...)

## Health Analysis

Get an overview of the overall health of the dataset. This is usually quick to compute and hopefully highlights some problems to focus on.



### Summary Plot

Provides a high level summary of the dataset health.

**Watch out for:**

* too few numeric values
* high % of missing values
* high % of duplicate values
* high % of duplicate columns 

In [60]:
sum1, plot1 = eda.get_data_health_summary(google_df, return_plot=True)
sum2, plot2 = eda.get_data_health_summary(pro_manu_df, return_plot=True)
sum3, plot3 = eda.get_data_health_summary(sales_df, return_plot=True)
sum4, plot4 = eda.get_data_health_summary(soc_media_df, return_plot=True)
sum5, plot5 = eda.get_data_health_summary(theme_df, return_plot=True)
sum6, plot6 = eda.get_data_health_summary(theme_pro_df, return_plot=True)

display_as_tabs([('google', plot1), ('product_manufacturer_list', plot2),
                 ('sales_data', plot3), ('social_media_data', plot4),
                ('Theme_list', plot5), ('Theme_product_list', plot6)])

Tabs
    [0] HoloViews(Layout, name='google')
    [1] HoloViews(Layout, name='product_manufacturer_list...)
    [2] HoloViews(Layout, name='sales_data')
    [3] HoloViews(Layout, name='social_media_data')
    [4] HoloViews(Layout, name='Theme_list')
    [5] HoloViews(Layout, name='Theme_product_list')

**Dev NOTES**

<details>
1. Datatypes : We have both numeric and other types. The bulk of them seem to be numeric. `Numeric` is defined to be one of [float|int|date] and the rest are categorized as `Others`. A column is assumed to have `date` values if it has the string `date` in the column name.

2. The missing value plot seems to indicate missing values are not present but we do have them. 

3. We are looking for duplicate observations (rows in the data). The plot shows the % of rows that are an exact replica of another row (using `df.duplicated`)

4. We are looking for duplicate features (columns in the data).

</details>

### Missing Values summary

This provides an overall view focussing on amount of missing values in the dataset.

**Watch out for:**
* A few columns have significant number of missing values 
* Most columns have significant number of missing values


In [61]:
sum1, plot1 = eda.get_missing_values_summary(google_df, return_plot=True)
sum2, plot2 = eda.get_missing_values_summary(pro_manu_df, return_plot=True)
sum3, plot3 = eda.get_missing_values_summary(sales_df, return_plot=True)
sum4, plot4 = eda.get_missing_values_summary(soc_media_df, return_plot=True)
sum5, plot5 = eda.get_missing_values_summary(theme_df, return_plot=True)
sum6, plot6 = eda.get_missing_values_summary(theme_pro_df, return_plot=True)

display_as_tabs([('google', plot1), ('product_manufacturer_list', plot2),
                 ('sales_data', plot3), ('social_media_data', plot4),
                ('Theme_list', plot5), ('Theme_product_list', plot6)])

Tabs
    [0] HoloViews(Bars, name='google')
    [1] HoloViews(Bars, name='product_manufacturer_list...)
    [2] HoloViews(Bars, name='sales_data')
    [3] HoloViews(Bars, name='social_media_data')
    [4] HoloViews(Bars, name='Theme_list')
    [5] HoloViews(Bars, name='Theme_product_list')

**Dev notes:**

<details>
    
    * By default, the following are considered missing/NA values : `[np.Nan, pd.NaT, 'NA', None]`
    * additional values can be passed to tigerml (add_additional_na_values)
    * these are applied to all columns.
    
    * some of the above information can be learnt from the data discovery step (see discussion below)
    
</details>

In [62]:
sum1 = eda.get_duplicate_columns(google_df)
sum2 = eda.get_duplicate_columns(pro_manu_df)
sum3 = eda.get_duplicate_columns(sales_df)
sum4 = eda.get_duplicate_columns(soc_media_df)
sum5 = eda.get_duplicate_columns(theme_df)
sum6 = eda.get_duplicate_columns(theme_pro_df)

display_as_tabs([('google', sum1), ('product_manufacturer_list', sum2),
                 ('sales_data', sum3), ('social_media_data', sum4),
                ('Theme_list', sum5), ('Theme_product_list', sum6)])

Tabs
    [0] Markdown(str, name='google')
    [1] Markdown(str, name='product_manufacturer_list...)
    [2] Markdown(str, name='sales_data')
    [3] Markdown(str, name='social_media_data')
    [4] Markdown(str, name='Theme_list')
    [5] Markdown(str, name='Theme_product_list')

In [63]:
sum1 = eda.get_outliers(google_df)
sum2 = eda.get_outliers(pro_manu_df)
sum3 = eda.get_outliers(sales_df)
sum4 = eda.get_outliers(soc_media_df)
sum5 = eda.get_outliers(theme_df)
sum6 = eda.get_outliers(theme_pro_df)

display_as_tabs([('google', sum1), ('product_manufacturer_list', sum2),
                 ('sales_data', sum3), ('social_media_data', sum4),
                ('Theme_list', sum5), ('Theme_product_list', sum6)])

Tabs
    [0] DataFrame(name='google', value=              ('Data Shape...)
    [1] Markdown(str, name='product_manufacturer_list...)
    [2] DataFrame(name='sales_data', value=                     ('Dat...)
    [3] DataFrame(name='social_media_data', value=            ('Data Shape:(...)
    [4] Markdown(str, name='Theme_list')
    [5] DataFrame(name='Theme_product_list', value=          ('Data Shape:(91...)

## Health Analysis report

Generate a report that has all the above data in a single html. This could be useful to submit to a client

In [64]:
from ta_lib.reports.api import summary_report

summary_report(google_df, './google.html')
summary_report(pro_manu_df, './product_manufacturer_list.html')
summary_report(sales_df, './sales_data.html')
summary_report(soc_media_df, './social_media_data.html')
summary_report(theme_df, './Theme_list.html')
summary_report(theme_pro_df, './Theme_product_list.html')

# Dataset Merging 

* We will merge the dataset which will speically focus on our client



In [65]:
# Merging the Sales with the  theme  after merfing it with the theme product Dataset
sa_tp = pd.merge(sales_df , theme_pro_df , left_on= 'product_id',right_on = "PRODUCT_ID")
sa_th = pd.merge(sa_tp , theme_df , on = 'CLAIM_ID')
sa_th.head()

,system_calendar_key_N,product_id,sales_dollars_value,sales_units_value,sales_lbs_value,PRODUCT_ID,CLAIM_ID,Claim Name
0,20160109,1,13927.0,934,18680,1,0,No Claim
1,20160123,1,12628.0,878,17564,1,0,No Claim
2,20160206,1,11379.0,810,16200,1,0,No Claim
3,20160130,1,11568.0,821,16424,1,0,No Claim
4,20160213,1,10959.0,784,15682,1,0,No Claim


In [66]:
# Getting the Data only focus on our client A
vendor = pd.merge(sa_th , pro_manu_df , on = 'PRODUCT_ID')
vendor.head()

,system_calendar_key_N,product_id,sales_dollars_value,sales_units_value,sales_lbs_value,PRODUCT_ID,CLAIM_ID,Claim Name,Vendor
0,20160109,1,13927.0,934,18680,1,0,No Claim,Others
1,20160123,1,12628.0,878,17564,1,0,No Claim,Others
2,20160206,1,11379.0,810,16200,1,0,No Claim,Others
3,20160130,1,11568.0,821,16424,1,0,No Claim,Others
4,20160213,1,10959.0,784,15682,1,0,No Claim,Others


In [67]:
vendor.groupby(['Vendor','Claim Name'])['sales_dollars_value'].sum()

Vendor         Claim Name              
A              No Claim                    2.352178e+10
               american gumbo              2.573988e+06
               american southwest style    4.557530e+08
               apple cinnamon              1.241048e+09
               beans                       9.654130e+05
                                               ...     
Private Label  pizza                       2.084414e+08
               pollock                     5.022839e+08
               red raspberry               5.137100e+04
               salmon                      2.654635e+08
               soy foods                   3.988618e+07
Name: sales_dollars_value, Length: 197, dtype: float64

In [68]:
# Creating a seperate Dataset for Clietn A
Client_A= vendor[vendor.Vendor =='A']
Client_A.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 811370 entries, 9459 to 7760500
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   system_calendar_key_N  811370 non-null  int64  
 1   product_id             811370 non-null  int64  
 2   sales_dollars_value    811370 non-null  float64
 3   sales_units_value      811370 non-null  int64  
 4   sales_lbs_value        811370 non-null  int64  
 5   PRODUCT_ID             811370 non-null  int64  
 6   CLAIM_ID               811370 non-null  int64  
 7   Claim Name             811370 non-null  object 
 8   Vendor                 811370 non-null  object 
dtypes: float64(1), int64(6), object(2)
memory usage: 61.9+ MB


In [69]:
Client_A.head()

,system_calendar_key_N,product_id,sales_dollars_value,sales_units_value,sales_lbs_value,PRODUCT_ID,CLAIM_ID,Claim Name,Vendor
9459,20160109,587,156606.0,40400,161603,587,0,No Claim,A
9460,20160123,587,162052.0,41781,167125,587,0,No Claim,A
9461,20160206,587,167919.0,43560,174239,587,0,No Claim,A
9462,20160130,587,162358.0,42077,168307,587,0,No Claim,A
9463,20160213,587,157831.0,40814,163256,587,0,No Claim,A


In [70]:
google_search=google_df.groupby("Claim_ID")["searchVolume"].mean()

In [71]:
google_search

Claim_ID
8      3759.708522
39      232.585921
40       66.682927
65      884.715996
75     1057.193137
          ...     
981      23.925926
982       6.250000
984     439.835294
985      31.500000
999     708.423679
Name: searchVolume, Length: 160, dtype: float64

In [72]:
media_post=soc_media_df.groupby("Theme Id")["total_post"].mean()

In [73]:
media_post

Theme Id
8.0      329.786462
15.0       6.419900
26.0       0.203692
38.0       4.008615
39.0       4.382769
            ...    
982.0     59.853538
983.0    146.875077
984.0     25.801846
985.0      5.382979
999.0     37.273467
Name: total_post, Length: 193, dtype: float64

In [74]:
Client_A["totalposts"]=Client_A['CLAIM_ID'].map(dict(media_post))
Client_A["searchvolume"]=Client_A["CLAIM_ID"].map(dict(google_search))


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [75]:
Client_A["searchvolume"].fillna(Client_A["searchvolume"].mean(),inplace=True)
Client_A["totalposts"].fillna(Client_A["totalposts"].mean(),inplace=True)


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [76]:
Client_A.head()

,system_calendar_key_N,product_id,sales_dollars_value,sales_units_value,sales_lbs_value,PRODUCT_ID,CLAIM_ID,Claim Name,Vendor,totalposts,searchvolume
9459,20160109,587,156606.0,40400,161603,587,0,No Claim,A,185.292605,2554.6548
9460,20160123,587,162052.0,41781,167125,587,0,No Claim,A,185.292605,2554.6548
9461,20160206,587,167919.0,43560,174239,587,0,No Claim,A,185.292605,2554.6548
9462,20160130,587,162358.0,42077,168307,587,0,No Claim,A,185.292605,2554.6548
9463,20160213,587,157831.0,40814,163256,587,0,No Claim,A,185.292605,2554.6548


In [77]:
Client_A['system_calendar_key_N'] = pd.to_datetime(Client_A['system_calendar_key_N'], format='%Y%m%d')


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [78]:
Client_A['Month'] = pd.DatetimeIndex(Client_A['system_calendar_key_N']).month
Client_A['year'] = pd.DatetimeIndex(Client_A['system_calendar_key_N']).year
Client_A['day'] = pd.DatetimeIndex(Client_A['system_calendar_key_N']).day
Client_A['week'] = pd.DatetimeIndex(Client_A['system_calendar_key_N']).week


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [79]:
Client_A = Client_A.drop(columns= 'system_calendar_key_N' )

In [80]:
Client_A.head()

,product_id,sales_dollars_value,sales_units_value,sales_lbs_value,PRODUCT_ID,CLAIM_ID,Claim Name,Vendor,totalposts,searchvolume,Month,year,day,week
9459,587,156606.0,40400,161603,587,0,No Claim,A,185.292605,2554.6548,1,2016,9,1
9460,587,162052.0,41781,167125,587,0,No Claim,A,185.292605,2554.6548,1,2016,23,3
9461,587,167919.0,43560,174239,587,0,No Claim,A,185.292605,2554.6548,2,2016,6,5
9462,587,162358.0,42077,168307,587,0,No Claim,A,185.292605,2554.6548,1,2016,30,4
9463,587,157831.0,40814,163256,587,0,No Claim,A,185.292605,2554.6548,2,2016,13,6


In [81]:
Client_A =Client_A.drop(columns={'product_id','sales_units_value','PRODUCT_ID',
                             'Claim Name','Vendor'})

In [82]:
Client_A.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 811370 entries, 9459 to 7760500
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   sales_dollars_value  811370 non-null  float64
 1   sales_lbs_value      811370 non-null  int64  
 2   CLAIM_ID             811370 non-null  int64  
 3   totalposts           811370 non-null  float64
 4   searchvolume         811370 non-null  float64
 5   Month                811370 non-null  int64  
 6   year                 811370 non-null  int64  
 7   day                  811370 non-null  int64  
 8   week                 811370 non-null  int64  
dtypes: float64(3), int64(6)
memory usage: 61.9 MB


In [83]:
# Saving the Dataset
save_dataset(context, Client_A, 'cleaned/sales')

In [117]:
sum9, plot9 = eda.get_data_health_summary(Client_A, return_plot=True)
display_as_tabs([('Client_A', plot9)])

Tabs
    [0] HoloViews(Layout, name='Client_A')

In [112]:
sum9, plot9 = eda.get_missing_values_summary(Client_A, return_plot=True)
display_as_tabs([('Client_A', plot9)])

Tabs
    [0] HoloViews(Bars, name='Client_A')

# Train Test Split

In [84]:
from sklearn.model_selection import StratifiedShuffleSplit , train_test_split
from ta_lib.core.api import custom_train_test_split  # helper function to customize splitting
from scripts import *


sales_df_train, sales_df_test = train_test_split(Client_A, test_size = 0.3 , random_state =102)

In [85]:
target_col = "sales_dollars_value"

train_X, train_y = (
    sales_df_train
    
    # split the dataset to train and test
    .get_features_targets(target_column_names=target_col)
)
save_dataset(context, train_X, 'train/sales/features')
save_dataset(context, train_y, 'train/sales/target')


test_X, test_y = (
    sales_df_test
    
    # split the dataset to train and test
    .get_features_targets(target_column_names=target_col)
)
save_dataset(context, test_X, 'test/sales/features')
save_dataset(context, test_y, 'test/sales/target')

In [86]:
from xgboost import XGBRegressor
from ta_lib.regression.api import SKLStatsmodelOLS
from ta_lib.regression.api import RegressionReport,RegressionComparison
from ta_lib.data_processing.api import Outlier

In [110]:
from xgboost import XGBRegressor
from ta_lib.regression.api import SKLStatsmodelOLS
from ta_lib.regression.api import RegressionReport,RegressionComparison
from ta_lib.data_processing.api import Outlier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import numpy as np

#  Modelling

## 1. Modelling - OLS

In [88]:
import statsmodels.api as sm
import pandas as pd
# fitting the model
result = sm.OLS(train_y,train_X).fit()
 
# printing the summary table
print(result.summary())

                                 OLS Regression Results                                 
Dep. Variable:     sales_dollars_value   R-squared (uncentered):                   0.776
Model:                             OLS   Adj. R-squared (uncentered):              0.776
Method:                  Least Squares   F-statistic:                          2.453e+05
Date:                 Sun, 29 May 2022   Prob (F-statistic):                        0.00
Time:                         17:23:21   Log-Likelihood:                     -7.0878e+06
No. Observations:               567959   AIC:                                  1.418e+07
Df Residuals:                   567951   BIC:                                  1.418e+07
Df Model:                            8                                                  
Covariance Type:             nonrobust                                                  
                      coef    std err          t      P>|t|      [0.025      0.975]
--------------------------

In [89]:
pred_ols = result.predict(test_X)

In [90]:
MSE_ols = mean_squared_error(test_y,pred_ols)
MSE_ols

4017012066.933946

In [91]:
RMSE =  mean_squared_error(test_y,pred_ols,squared=False)
RMSE

63379.90270530514

## 2. Modelling - Linear Regression

In [109]:
le = LinearRegression()
le.fit(train_X,train_y)

print(le.coef_)
print(le.intercept_)

train_pred = le.predict(train_X)
test_pred = le.predict(test_X)
  
rmse1 = np.sqrt(MSE(train_y, train_pred))
rmse2 = np.sqrt(MSE(test_y, test_pred))
print("train RMSE : % f" %(rmse1))
print("test RMSE : % f" %(rmse2))

r21=r2_score(train_y,train_pred)
r22=r2_score(test_y, test_pred)
print("train R2 score : % f" %(r21))
print("test R2 score : % f" %(r22))

mae1=mean_absolute_error(train_y, train_pred)
mae2=mean_absolute_error(test_y, test_pred)
print("train MAE : % f" %(mae1))
print("test MAE : % f" %(mae2))

le

[ 5.17733496e-01 -1.69603710e+01 -2.84194866e+01  6.13976996e-01
  2.71467367e+03 -1.47331577e+03  6.85509701e+01 -6.64348071e+02]
3003048.3671014044
train RMSE :  63607.570870
test RMSE :  63373.904359
train R2 score :  0.744214
test R2 score :  0.749638
train MAE :  33801.353551
test MAE :  33804.248647


LinearRegression()

In [126]:
le.score(test_X,test_y)

0.7496380416964934

## 3. Modelling - XGB regresser

In [108]:
from sklearn.metrics import mean_squared_error as MSE,r2_score,mean_absolute_error

model= XGBRegressor(booster='gblinear',n_estimators = 100,learning_rate=0.1,n_jobs=4)
model.fit(train_X, train_y)

print(model.coef_)
print(model.intercept_)

train_pred = model.predict(train_X)
test_pred = model.predict(test_X)
  
rmse1 = np.sqrt(MSE(train_y, train_pred))
rmse2 = np.sqrt(MSE(test_y, test_pred))
print("train RMSE : % f" %(rmse1))
print("test RMSE : % f" %(rmse2))

r21=r2_score(train_y,train_pred)
r22=r2_score(test_y, test_pred)
print("train R2 score : % f" %(r21))
print("test R2 score : % f" %(r22))

mae1=mean_absolute_error(train_y, train_pred)
mae2=mean_absolute_error(test_y, test_pred)
print("train MAE : % f" %(mae1))
print("test MAE : % f" %(mae2))

model

[ 5.18524e-01 -1.18692e+01 -6.69246e+00 -8.40068e-02  3.61109e+02
  5.65916e+00  1.07832e+02 -4.86558e+01]
[14797.4]
train RMSE :  63662.404755
test RMSE :  63422.588597
train R2 score :  0.743773
test R2 score :  0.749253
train MAE :  33698.004436
test MAE :  33698.329071


XGBRegressor(base_score=0.5, booster='gblinear', colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=-1, importance_type='gain', interaction_constraints=None,
             learning_rate=0.1, max_delta_step=None, max_depth=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=4, num_parallel_tree=None, random_state=0,
             reg_alpha=0, reg_lambda=0, scale_pos_weight=1, subsample=None,
             tree_method=None, validate_parameters=1, verbosity=None)

In [120]:
np.mean(np.abs((test_y -test_pred)/test_y))

inf

In [124]:
model.score(train_X,train_y)

0.7437732188846103